In [9]:
import pandas as pd
import ast
import numpy as np
import scipy.stats as stats 
import statsmodels.stats.api as sms
import seaborn as sns
import matplotlib.pyplot as plt
import os
from statsmodels.stats.multitest import multipletests

# Read the JSON blob formats

In [4]:
os.chdir('../')
os.getcwd()

'/Users/xehu/Desktop/Team Process Mapping/tpm-horse-race-modeling'

In [7]:
EVAL_OUTPUT_PATH = "./model_eval_cache/evaluation_outputs/stageId_cumulative/task_attr_ElasticNetEmpirical_score_10_evaluation"

In [8]:
results = pd.DataFrame([ast.literal_eval(x.strip()) for x in open(EVAL_OUTPUT_PATH, "r").readlines()])
results

,q^2,iteration
0,0.224499,0
1,0.224495,1
2,0.224491,2
3,0.224497,3
4,0.224499,4
5,0.224495,5
6,0.224492,6
7,0.224492,7
8,0.224494,8
9,0.224497,9


# Plot Means with Confidence Intervans and T-Tests
Create a bar plot to compare the Q^2 for different models.

In [ ]:
def plot_means_with_confidence_intervals_and_ttests(evaluation_output_list, save_path, title_appendix = "", confidence_level=0.95, alpha=0.05):
	# Calculate means and confidence intervals
	means = [np.mean(observation) for observation in evaluation_output_list]
	errors = [(sms.DescrStatsW(observation).tconfint_mean()[1] - sms.DescrStatsW(observation).tconfint_mean()[0]) / 2. for observation in observation_lists]
	colors = plt.cm.tab20(np.arange(len(labels)))

	# Plot the bar graph with error bars
	plt.figure(figsize=(12, 8))
	plt.bar(range(len(means)), means, yerr=errors, align='center', alpha=0.7, ecolor='black', color=colors, capsize=10)

	# Add labels and title
	labels =    # ... TODO: Auto-generate labels

	plt.xticks(range(len(means)), labels, rotation=45, ha="right")  # Rotate x-axis labels by 45 degrees
	plt.ylabel('Prediction Q^2', size = 16)
	plt.title('Predictive Power of Models; Pairwise t-test with B-H Correction' + title_appendix, size = 18)

	# Perform pairwise t-tests
	HEIGHT_MULTIPLIER = 0.1

	p_values = []
	i_s = []
	j_s = []

	line_height = np.mean(max(observation_lists)) * 0.1
	for i in range(len(observation_lists)):
		for j in range(i + 1, len(observation_lists)):
			t_stat, p_value = stats.ttest_ind(observation_lists[i], observation_lists[j])
			p_values.append(p_value)
			i_s.append(i)
			j_s.append(j)

	# Correct p-values for multiple comparisons using Benjamini-Hochberg procedure
	_, p_values_corrected, _, _ = multipletests(p_values, alpha=alpha, method='fdr_bh')

	# Draw horizontal bar between compared groups by using the corrected p-value
	for k in range(len(p_values_corrected)):

		p_value = p_values_corrected[k]
		i = i_s[k]
		j = j_s[k]

		# if p_value < alpha:
		if p_value >= alpha: ### plot the lines *only if n.s.*, as most are significant
			line_y = max(means) + max(errors) + np.mean(max(observation_lists)) * 0.03 + (i + j) * line_height * HEIGHT_MULTIPLIER  # Adjust the multiplier for better spacing
			plt.plot([i, j], [line_y, line_y], color='black')

			# Display significance stars based on p-value
			if p_value < 0.001:
				significance_label = '***'
			elif p_value < 0.01:
				significance_label = '**'
			elif p_value < 0.05:
				significance_label = '*'
			else:
				significance_label = 'n.s.'

			# Display significance labels on the plot
			plt.text((i + j) / 2, line_y + np.mean(max(observation_lists)) * 0.025, significance_label, ha='center', va='center')

	# Show the plot
	plt.savefig(save_path+".svg")
	plt.savefig(save_path+".png")